In [ ]:
import cv2  #gerekli olan kütüphaneleri (paketleri) tanımlarız
import numpy as np

cap  = cv2.VideoCapture(0)  #görüntüyü webcamden alırız

ret,frame = cap.read()  #bir tane frame okuruz

if ret == False: #eğer ki okuyamazsak uyarı mesajı göndeririz
    print('Uyarı')

face_cascade = cv2.CascadeClassifier('D:/examples_forDeepLearning/ObjectTracking/meanShiftAlgorithm/haarcascade_frontalface_default.xml')  #cascade

face_rects = face_cascade.detectMultiScale(frame) 

(face_x,face_y,w,h) = tuple(face_rects[0])


track_window = (face_x,face_y,w,h)  #meanshift algoritması girdisi

roi = frame[face_y:face_y + h,face_x:face_x + w] #tespit ettiğimiz kutucuğun içerisi yani roi = face

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)  #bgr formatından hsv formatına çeviriyoruz

roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180]) #takip için histogram gerekli 
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)  #0 ile 255 arasına sıkıştırıyoruz
#takip için gerekli olan durdurma kriterleri
#count = hesaplanacak maksimum öge sayısı 
#epsilon = değişiklik

term_crit = (cv2.TERM_CRITERIA_EPS or cv2.TERM_CRITERIA_COUNT,5 ,1)  # 5 yineleme veya 1 tane epsilon

while True:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    if ret == False:
        break
    #histogramı bir görüntüde bulmak için kullanıyoruz ve böylece eşleme gerçekleşip takip sürüyor
    #piksel karşılaştırma yapılıyor 
    dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)   

    ret,track_window = cv2.meanShift(dst,track_window,term_crit) #takip algoritmasını kullanıyoruz ve içine gerekli parametreleri giriyoruz(durma kriteri)
    x,y,w,h = track_window # 'track_window' bize takip ettiği nesnenin yeni konumunu döndürecek(ayrıca GENİŞLİK VE yükseklikte dönderiyor)

    img2 = cv2.rectangle(frame,(x,y),(x+w, y+h),(0,0,255),4) #kırmızı bir bounding box çiziyoruz

    cv2.imshow("Takip",img2)  

    if cv2.waitKey(1) == ord('q'): # 'q' tuşuna basılırsa döngüyü kır
        break
cap.release() #frameleri serbest bırak
cv2.destroyAllWindows() #tüm pencereleri kapatd



